In [1]:
#Within this project I devaloped a simple algorithmic trading model which uses sandbox api data to devalop a spreadsheet
#of the amount of each share to purchase within the S&P 500 based on the size of the portfolio

In [2]:
import numpy as np
import pandas as pd
import requests as rq
import xlsxwriter
import math

In [3]:
stocks=pd.read_csv('sp_500_stocks.csv')

In [4]:
from secrets import IEX_CLOUD_API_TOKEN

In [5]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data=rq.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'NaN', 'calculationPrice': 'close', 'open': 128.38, 'openTime': 1626743319666, 'openSource': 'NaN', 'close': 127.47, 'closeTime': 1689606513382, 'closeSource': 'NaN', 'high': 133.58, 'highTime': 1669972276669, 'highSource': 'NaN', 'low': 129.53, 'lowTime': 1699189863382, 'lowSource': 'NaN', 'latestPrice': 131.92, 'latestSource': 'Close', 'latestTime': 'May 14, 2021', 'latestUpdate': 1700296787879, 'latestVolume': 84030755, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 132.1, 'delayedPriceTime': 1641004198094, 'oddLotDelayedPrice': 129.75, 'oddLotDelayedPriceTime': 1676812213773, 'extendedPrice': 133.66, 'extendedChange': 0.35, 'extendedChangePercent': 0.0028, 'extendedPriceTime': 1681463843944, 'previousClose': 125.36, 'previousVolume': 105897642, 'change': 2.58, 'changePercent': 0.02042, 'volume': 83636843, 'iexMarketPercent': None, 'iexVolume': None, 'avgTotalVolume': 950329

In [6]:
price=data['latestPrice']
market_cap=data['marketCap']

In [7]:
my_columns={'Ticker','Price','Market Cap', 'Number of Shares to buy'}
final_dataframe=pd.DataFrame(columns=my_columns)
final_dataframe

,Price,Number of Shares to buy,Market Cap,Ticker


In [8]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
        
    ],
    index=my_columns
    
    ),
    ignore_index=True

)

,Price,Number of Shares to buy,Market Cap,Ticker
0,AAPL,131.92,2167958957087,N/A


In [9]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [10]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = rq.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Price,Number of Shares to buy,Market Cap,Ticker
0,A,134.03,40352116189,N/A
1,AAL,23.30,14741779925,N/A
2,AAP,202.85,13901241535,N/A
3,AAPL,130.67,2220379707513,N/A
4,ABBV,119.38,208179867860,N/A
...,...,...,...,...
500,YUM,124.09,35601046106,N/A
501,ZBH,171.53,36579750943,N/A
502,ZBRA,501.12,26397798343,N/A
503,ZION,61.90,10046115939,N/A


In [ ]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

In [ ]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

In [ ]:
writer=pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades', index=False)


In [ ]:
background_color='#0a0a23'
font_color='#ffffff'

string_format=writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format=writer.book.add_format(
    {
        'num_format': '$0.00',
        'bg_color': background_color,
        'border': 1
    }
)
integer_format=writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)


In [ ]:
#writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
#writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
#writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
#writer.sheets['Recommended Trades'].set_column('D:D',18,string_format)
#writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
#writer.sheets['Recommended Trades'].write('B1', 'Stock Price', string_format)
#writer.sheets['Recommended Trades'].write('C1', 'Market Capitilization', string_format)
#writer.sheets['Recommended Trades'].write('D1', 'Number of shares to Buy', string_format)

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [ ]:
writer.save()